### Project Overview

Customer churn—the phenomenon where customers discontinue their subscription to a service—poses a significant challenge for subscription-based businesses. Accurately predicting which customers are likely to leave enables companies to proactively engage at-risk customers, optimize retention strategies, and ultimately improve profitability. This project aims to develop a predictive model using machine learning techniques such as logistic regression and random forests, combined with thoughtful feature engineering, to identify customers at high risk of churn. The insights derived will inform actionable business recommendations and a cost-benefit analysis to guide retention efforts

### Business Understanding
For subscription-based businesses, retaining existing customers is often more cost-effective than acquiring new ones. High churn rates can signal underlying issues with customer satisfaction, product fit, or competitive pressures, directly impacting revenue and growth. By understanding the drivers of churn and identifying at-risk customers, the business can:
Target retention campaigns more effectively (e.g., personalized offers, improved customer support)
Allocate resources efficiently to maximize return on investment in retention
Reduce lost revenue and improve customer lifetime value
The key business questions addressed in this project are:
Which customers are most likely to leave the service in the near future?
What are the main factors contributing to customer churn?
How can the business intervene to reduce churn, and what is the expected financial impact of these interventions?
The project will use historical customer data—including demographics, service usage, account information, and previous churn behavior—to build and evaluate predictive models. The final deliverables will include actionable recommendations and a cost-benefit analysis to support data-driven decision-making for customer retention.


### Import Libraries and Dataset

In [1]:
import pandas as pd
df = pd.read_csv("C:/Users/Administrator/Documents/Customer churn prediction project/WA_Fn-UseC_-Telco-Customer-Churn.csv")
print(df.head())

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

### Exploratory Data Analysis(EDA)

In [2]:
# Check for missing values
df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [3]:
# Check data types
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [4]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [5]:
df.duplicated().sum()

0

In [6]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

### Feature Engineering

In [11]:
# Create a new feature CustomerID_gender 
df['customerID_gender'] = df['customerID'] + '_' + df['gender']
print(df['customerID_gender'])

0       7590-VHVEG_Female
1         5575-GNVDE_Male
2         3668-QPYBK_Male
3         7795-CFOCW_Male
4       9237-HQITU_Female
              ...        
7038      6840-RESVB_Male
7039    2234-XADUH_Female
7040    4801-JZAZL_Female
7041      8361-LTMKD_Male
7042      3186-AJIEK_Male
Name: customerID_gender, Length: 7043, dtype: object


In [13]:
# CustomerID_PhoneService
df['customerID_PhoneService'] = df['customerID'] + '_' + df['PhoneService']
print(df['customerID_PhoneService'])

0        7590-VHVEG_No
1       5575-GNVDE_Yes
2       3668-QPYBK_Yes
3        7795-CFOCW_No
4       9237-HQITU_Yes
             ...      
7038    6840-RESVB_Yes
7039    2234-XADUH_Yes
7040     4801-JZAZL_No
7041    8361-LTMKD_Yes
7042    3186-AJIEK_Yes
Name: customerID_PhoneService, Length: 7043, dtype: object
